In [ ]:
# Setup
import os
import glob
import pandas as pd
import numpy as np
from typing import Tuple

from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import roc_auc_score, confusion_matrix, precision_recall_fscore_support
from scipy.stats import ks_2samp
from imblearn.over_sampling import RandomOverSampler

import numpy as np
from scipy.stats import ks_2samp
from sklearn.metrics import (
    roc_auc_score, confusion_matrix,
    precision_recall_fscore_support, log_loss
)
from sklearn.linear_model import LogisticRegression, LinearRegression
from tabpfn import TabPFNClassifier

from sklearn.model_selection import train_test_split



In [ ]:
from google.colab import files
uploaded = files.upload()


Saving customer_churn_telecom_services.csv to customer_churn_telecom_services.csv


In [ ]:
df_raw = pd.read_csv("customer_churn_telecom_services.csv")
df_raw.head()


,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,Female,0,Yes,No,1,No,No phone service,DSL,No,Yes,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,Male,0,No,No,34,Yes,No,DSL,Yes,No,Yes,No,No,No,One year,No,Mailed check,56.95,1889.50,No
2,Male,0,No,No,2,Yes,No,DSL,Yes,Yes,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,Male,0,No,No,45,No,No phone service,DSL,Yes,No,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,Female,0,No,No,2,Yes,No,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


In [ ]:
# Definir coluna alvo
target_col = "Churn"

# Normalizar o alvo para binário 0/1
df = df_raw.copy()
def to_binary(series):
    if series.dtype == 'O':
        return series.str.lower().map({'yes':1,'sim':1,'true':1,'y':1,'1':1,'no':0,'nao':0,'false':0,'n':0,'0':0}).fillna(series)
    return series
df[target_col] = to_binary(df[target_col])
if df[target_col].dtype == 'O':
    # Se ainda for objeto, tente astype(int) após mapear os únicos
    uniques = df[target_col].unique()
    print('Warning: target still object. Uniques:', uniques)

# Organiza TotalCharges como float
if df['TotalCharges'].dtype == 'O':
    df['TotalCharges'] = pd.to_numeric(df['TotalCharges'].str.strip().replace('', np.nan), errors='coerce')
    print('TotalCharges converted to float. Missing:', df['TotalCharges'].isna().sum())

print('Class balance:', df[target_col].value_counts(dropna=False))
print('Missing per column:\n', df.isna().sum())


Class balance: Churn
0    5174
1    1869
Name: count, dtype: int64
Missing per column:
 gender               0
SeniorCitizen        0
Partner              0
Dependents           0
tenure               0
PhoneService         0
MultipleLines        0
InternetService      0
OnlineSecurity       0
OnlineBackup         0
DeviceProtection     0
TechSupport          0
StreamingTV          0
StreamingMovies      0
Contract             0
PaperlessBilling     0
PaymentMethod        0
MonthlyCharges       0
TotalCharges        11
Churn                0
dtype: int64


In [ ]:
# Remove linhas com NaN em TotalCharges
before = len(df)
df = df.dropna(subset=['TotalCharges']).reset_index(drop=True)
after = len(df)
print(f'Dropped {before - after} rows with NaN TotalCharges. New shape: {df.shape}')
print('Class balance after drop:', df[target_col].value_counts(dropna=False))

Dropped 11 rows with NaN TotalCharges. New shape: (7032, 20)
Class balance after drop: Churn
0    5163
1    1869
Name: count, dtype: int64


In [ ]:
# Pré processamento: separar X e y
X = df.drop(columns=[target_col])
y = df[target_col].astype(int)

categorical_cols = [c for c in X.columns if X[c].dtype == 'O']
numeric_cols = [c for c in X.columns if c not in categorical_cols]
print('Categorical:', categorical_cols)
print('Numeric:', numeric_cols)

numeric_pipeline = Pipeline(steps=[
    ('impute', 'passthrough'),
])

categorical_pipeline = Pipeline(steps=[
    ('onehot', OneHotEncoder(handle_unknown='ignore', sparse_output=False))
])


Categorical: ['gender', 'Partner', 'Dependents', 'PhoneService', 'MultipleLines', 'InternetService', 'OnlineSecurity', 'OnlineBackup', 'DeviceProtection', 'TechSupport', 'StreamingTV', 'StreamingMovies', 'Contract', 'PaperlessBilling', 'PaymentMethod']
Numeric: ['SeniorCitizen', 'tenure', 'MonthlyCharges', 'TotalCharges']


In [ ]:
# Supondo que df já foi carregado e a coluna alvo é 'Churn'
df_class1 = df[df['Churn'] == 0]  # não churn
df_class2 = df[df['Churn'] == 1]  # churn

print("Classe 1 (não churn):", len(df_class1))
print("Classe 2 (churn):", len(df_class2))

total = len(df_class1) + len(df_class2)
print(f"Classe 1: {len(df_class1)} ({len(df_class1)/total:.2%})")
print(f"Classe 2: {len(df_class2)} ({len(df_class2)/total:.2%})")


Classe 1 (não churn): 5163
Classe 2 (churn): 1869
Classe 1: 5163 (73.42%)
Classe 2: 1869 (26.58%)


In [ ]:
# Para classe 1 (maioria)
train_c1, temp_c1 = train_test_split(df_class1, test_size=0.5, random_state=42, stratify=None)
val_c1, test_c1 = train_test_split(temp_c1, test_size=0.5, random_state=42)

# Para classe 2 (minoritária)
train_c2, temp_c2 = train_test_split(df_class2, test_size=0.5, random_state=42, stratify=None)
val_c2, test_c2 = train_test_split(temp_c2, test_size=0.5, random_state=42)


In [ ]:
from sklearn.utils import resample

# Balancear treino
train_c2_bal = resample(train_c2, replace=True, n_samples=len(train_c1), random_state=42)
val_c2_bal = resample(val_c2, replace=True, n_samples=len(val_c1), random_state=42)

# Teste fica sem repetição


In [ ]:
train_df = pd.concat([train_c1, train_c2_bal]).sample(frac=1, random_state=42).reset_index(drop=True)
val_df = pd.concat([val_c1, val_c2_bal]).sample(frac=1, random_state=42).reset_index(drop=True)
test_df = pd.concat([test_c1, test_c2]).sample(frac=1, random_state=42).reset_index(drop=True)


In [ ]:
X_train, y_train = train_df.drop(columns=['Churn']), train_df['Churn']
X_val, y_val = val_df.drop(columns=['Churn']), val_df['Churn']
X_test, y_test = test_df.drop(columns=['Churn']), test_df['Churn']


In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

# 1️⃣ Ajusta (fit) apenas com os dados de treinamento
scaler.fit(X_train[numeric_cols])

# 2️⃣ Aplica (transform) a mesma escala em todos os conjuntos
X_train[numeric_cols] = scaler.transform(X_train[numeric_cols])
X_val[numeric_cols]   = scaler.transform(X_val[numeric_cols])
X_test[numeric_cols]  = scaler.transform(X_test[numeric_cols])


In [ ]:
print("Tamanhos dos conjuntos:")
print("Treinamento:", len(train_df))
print("Validação:", len(val_df))
print("Teste:", len(test_df))


Tamanhos dos conjuntos:
Treinamento: 5162
Validação: 2582
Teste: 1759


In [ ]:
print("\nDistribuição das classes:")

print("\nTreino:")
print(y_train.value_counts())
print(y_train.value_counts(normalize=True))

print("\nValidação:")
print(y_val.value_counts())
print(y_val.value_counts(normalize=True))

print("\nTeste:")
print(y_test.value_counts())
print(y_test.value_counts(normalize=True))



Distribuição das classes:

Treino:
Churn
0    2581
1    2581
Name: count, dtype: int64
Churn
0    0.5
1    0.5
Name: proportion, dtype: float64

Validação:
Churn
0    1291
1    1291
Name: count, dtype: int64
Churn
0    0.5
1    0.5
Name: proportion, dtype: float64

Teste:
Churn
0    1291
1     468
Name: count, dtype: int64
Churn
0    0.73394
1    0.26606
Name: proportion, dtype: float64


In [ ]:
# One-Hot Encoding
X_train = pd.get_dummies(X_train, drop_first=True)
X_val   = pd.get_dummies(X_val, drop_first=True)
X_test  = pd.get_dummies(X_test, drop_first=True)

# Alinhar colunas
X_train, X_val = X_train.align(X_val, join='left', axis=1, fill_value=0)
X_train, X_test = X_train.align(X_test, join='left', axis=1, fill_value=0)


In [ ]:
bool_cols = X_train.select_dtypes(include='bool').columns

X_train[bool_cols] = X_train[bool_cols].astype(int)
X_val[bool_cols]   = X_val[bool_cols].astype(int)
X_test[bool_cols]  = X_test[bool_cols].astype(int)


In [ ]:
print("Shapes finais:")
print("X_train:", X_train.shape, "y_train:", y_train.shape)
print("X_val  :", X_val.shape,   "y_val  :", y_val.shape)
print("X_test :", X_test.shape,  "y_test :", y_test.shape)


Shapes finais:
X_train: (5162, 30) y_train: (5162,)
X_val  : (2582, 30) y_val  : (2582,)
X_test : (1759, 30) y_test : (1759,)


In [ ]:
X_train.head()


,SeniorCitizen,tenure,MonthlyCharges,TotalCharges,gender_Male,Partner_Yes,Dependents_Yes,PhoneService_Yes,MultipleLines_No phone service,MultipleLines_Yes,...,StreamingTV_No internet service,StreamingTV_Yes,StreamingMovies_No internet service,StreamingMovies_Yes,Contract_One year,Contract_Two year,PaperlessBilling_Yes,PaymentMethod_Credit card (automatic),PaymentMethod_Electronic check,PaymentMethod_Mailed check
0,-0.486522,1.579406,-1.326839,-0.038879,1,0,0,0,1,0,...,0,0,0,0,1,0,0,1,0,0
1,-0.486522,0.119458,0.459904,0.261448,1,0,0,1,0,1,...,0,0,0,0,0,0,1,0,1,0
2,-0.486522,-1.006788,-0.754874,-0.856418,0,0,0,1,0,0,...,0,0,0,0,0,0,1,0,0,1
3,-0.486522,-0.005681,-0.118973,-0.121568,1,1,1,1,0,1,...,0,0,0,1,0,0,0,0,1,0
4,2.055407,-0.339383,0.200705,-0.276277,1,1,0,1,0,0,...,0,0,0,0,0,0,1,0,1,0


In [ ]:
X_train = X_train.astype(np.float32)
X_val   = X_val.astype(np.float32)
X_test  = X_test.astype(np.float32)


In [ ]:
print(X_train.shape[1])

30


In [ ]:
X_train.head()


,SeniorCitizen,tenure,MonthlyCharges,TotalCharges,gender_Male,Partner_Yes,Dependents_Yes,PhoneService_Yes,MultipleLines_No phone service,MultipleLines_Yes,...,StreamingTV_No internet service,StreamingTV_Yes,StreamingMovies_No internet service,StreamingMovies_Yes,Contract_One year,Contract_Two year,PaperlessBilling_Yes,PaymentMethod_Credit card (automatic),PaymentMethod_Electronic check,PaymentMethod_Mailed check
0,-0.486522,1.579406,-1.326839,-0.038879,1.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
1,-0.486522,0.119458,0.459904,0.261448,1.0,0.0,0.0,1.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
2,-0.486522,-1.006788,-0.754874,-0.856418,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
3,-0.486522,-0.005681,-0.118973,-0.121568,1.0,1.0,1.0,1.0,0.0,1.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
4,2.055407,-0.339383,0.200705,-0.276277,1.0,1.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0


# Utilizando o Modelo Pré-Treinado!

O código implementa um pipeline completo de classificação binária baseado no TabPFN v2, com o objetivo de servir como um baseline forte e eficiente para dados tabulares. A abordagem adota uma separação explícita em conjuntos de treino, validação e teste, permitindo avaliar de forma consistente tanto a capacidade de discriminação do modelo quanto a qualidade probabilística de suas previsões.

Inicialmente, os dados são convertidos de estruturas pandas para numpy, garantindo compatibilidade com o TabPFN e maior eficiência computacional. As variáveis preditoras são convertidas para float32, reduzindo consumo de memória e favorecendo execução em GPU, enquanto os rótulos são convertidos para inteiros binários. Assume-se, portanto, que as variáveis de entrada já estejam previamente tratadas e codificadas de forma numérica.

A avaliação do modelo utiliza um conjunto abrangente de métricas. A capacidade de discriminação é medida por meio do AUROC e da estatística KS (Kolmogorov–Smirnov), que avaliam o quão bem o modelo separa as distribuições de probabilidades associadas às classes positiva e negativa. Em paralelo, a qualidade das probabilidades preditas é analisada com Cross-Entropy (log-loss) e erro quadrático médio (MSE), métricas sensíveis à calibração e à confiança das previsões. Além disso, métricas dependentes de decisão — precisão, revocação, F1-score e matriz de confusão — são reportadas para permitir uma análise mais operacional do modelo.

Um aspecto central do pipeline é a definição do threshold de decisão. Em vez de adotar arbitrariamente o valor 0,5, o ponto de corte é escolhido no conjunto de validação como aquele que maximiza a estatística KS. Esse procedimento fornece um critério estatístico consistente para separar as classes e reduz a arbitrariedade na escolha do threshold. O valor obtido na validação é então mantido fixo para a avaliação nos conjuntos de treino e teste, evitando viés otimista na análise de desempenho.

O código também incorpora um módulo de calibração de probabilidades, permitindo comparar três cenários: ausência de calibração, calibração por Platt scaling e calibração focada em minimizar o MSE. O Platt scaling ajusta uma regressão logística sobre o logit das probabilidades, visando melhorar a log-loss e a interpretação probabilística. Já a calibração por regressão linear busca reduzir o erro quadrático médio, produzindo probabilidades mais próximas dos rótulos observados em média. Essa estrutura permite analisar empiricamente o trade-off entre boa discriminação e boa calibração das probabilidades.

O modelo TabPFN é configurado para execução em GPU quando disponível, com parâmetros que priorizam eficiência computacional. O uso de apenas um estimador (n_estimators = 1) reduz significativamente o tempo de execução, tornando o pipeline adequado para experimentação rápida e comparação com outros modelos. A opção de cache (fit_with_cache) acelera chamadas repetidas de inferência, enquanto a desativação das restrições de pretraining garante que o modelo possa ser executado mesmo fora do regime original de treinamento, ainda que isso deva ser interpretado com cautela.

No fluxo final, o TabPFN é treinado exclusivamente com o conjunto de treino, as probabilidades são geradas para treino, validação e teste, e a calibração é aplicada de forma consistente antes da escolha do threshold. As métricas são então calculadas para cada subconjunto, permitindo avaliar generalização, estabilidade e possíveis sinais de sobreajuste. Dessa forma, o pipeline fornece uma base metodológica sólida para análise comparativa, permitindo não apenas medir desempenho preditivo, mas também discutir calibração, critérios de decisão e limitações inerentes ao uso de modelos pré-treinados em dados tabulares.

In [ ]:
!pip -q install -U huggingface_hub tabpfn
!hf auth login

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.0/521.0 kB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 554.0/554.0 kB 41.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.7/144.7 kB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 5.6 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.
transformers 4.57.3 requires huggingface-hub<1.0,>=0.34.0, but you have huggingface-hub 1.2.3 which is incompatible.

    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  

In [ ]:
!pip -q install tabpfn scikit-learn scipy numpy


In [ ]:
import torch
print("CUDA?", torch.cuda.is_available())
print(torch.cuda.get_device_name(0) if torch.cuda.is_available() else "sem GPU")


CUDA? True
Tesla T4


In [ ]:
# =========================
# Conversões (pandas -> numpy)
# =========================
def X_to_np(X):
    return X.to_numpy().astype(np.float32)

def y_to_np(y):
    return y.to_numpy().astype(int)


# =========================
# Métricas
# =========================
def ks_stat(y, p):
    return ks_2samp(p[y == 1], p[y == 0]).statistic

def best_threshold_by_ks(y, p):
    p_sorted = np.sort(p)
    p1 = np.sort(p[y == 1])
    p0 = np.sort(p[y == 0])

    cdf1 = np.searchsorted(p1, p_sorted, side="right") / max(1, len(p1))
    cdf0 = np.searchsorted(p0, p_sorted, side="right") / max(1, len(p0))

    diffs = np.abs(cdf1 - cdf0)
    idx = np.argmax(diffs)
    return float(p_sorted[idx]), float(diffs[idx])

def mse(y, p):
    return float(np.mean((p - y) ** 2))

def ce(y, p):
    p = np.clip(p, 1e-7, 1 - 1e-7)
    return float(log_loss(y, p))

def full_metrics(y, p, threshold):
    y_hat = (p >= threshold).astype(int)

    cm = confusion_matrix(y, y_hat)
    prec, rec, f1, _ = precision_recall_fscore_support(
        y, y_hat, average="binary", zero_division=0
    )

    return {
        "KS": ks_stat(y, p),
        "AUROC": roc_auc_score(y, p),
        "CrossEntropy": ce(y, p),
        "MSE": mse(y, p),
        "precision": prec,
        "recall": rec,
        "f1": f1,
        "confusion_matrix": cm
    }


# =========================
# Calibração (CE vs MSE)
# =========================
def logit(p):
    p = np.clip(p, 1e-7, 1 - 1e-7)
    return np.log(p / (1 - p))

class NoCalibrator:
    def fit(self, p, y): return self
    def predict(self, p): return p

class PlattCalibrator:   # melhora Cross-Entropy
    def __init__(self):
        self.clf = LogisticRegression(solver="lbfgs")
    def fit(self, p, y):
        self.clf.fit(logit(p).reshape(-1, 1), y)
        return self
    def predict(self, p):
        return self.clf.predict_proba(logit(p).reshape(-1, 1))[:, 1]

class MSECalibrator:     # minimiza MSE
    def __init__(self):
        self.reg = LinearRegression()
    def fit(self, p, y):
        self.reg.fit(p.reshape(-1, 1), y)
        return self
    def predict(self, p):
        return np.clip(self.reg.predict(p.reshape(-1, 1)), 0, 1)

def get_calibrator(kind):
    if kind == "none": return NoCalibrator()
    if kind == "platt": return PlattCalibrator()
    if kind == "mse": return MSECalibrator()
    raise ValueError("Use: none | platt | mse")


# =========================
# Pipeline TabPFN v2
# =========================
def run_tabpfn2(
    X_train, y_train,
    X_val, y_val,
    X_test, y_test,
    calibrator="none",
    seed=42,
    force_cpu_ok=True
):
    import os

    if force_cpu_ok:
        os.environ["TABPFN_ALLOW_CPU_LARGE_DATASET"] = "1"

    Xtr, ytr = X_to_np(X_train), y_to_np(y_train)
    Xva, yva = X_to_np(X_val),   y_to_np(y_val)
    Xte, yte = X_to_np(X_test),  y_to_np(y_test)

    # ignore_pretraining_limits=True também libera o bloqueio
    model = TabPFNClassifier(
      device="cuda",               # usa GPU (se o runtime tiver GPU)
      random_state=seed,
      ignore_pretraining_limits=True,
      n_estimators=1,              # equivalente ao ensemble; 1 = bem mais rápido
      fit_mode="fit_with_cache",   # cache pra acelerar predict_proba
    )

    model.fit(Xtr, ytr)

    p_tr = model.predict_proba(Xtr)[:, 1]
    p_va = model.predict_proba(Xva)[:, 1]
    p_te = model.predict_proba(Xte)[:, 1]

    cal = get_calibrator(calibrator).fit(p_tr, ytr)
    p_tr = cal.predict(p_tr)
    p_va = cal.predict(p_va)
    p_te = cal.predict(p_te)

    t_star, ks_star = best_threshold_by_ks(yva, p_va)

    m_train = full_metrics(ytr, p_tr, t_star)
    m_val   = full_metrics(yva, p_va, t_star)
    m_test  = full_metrics(yte, p_te, t_star)

    print(f"\n[TabPFN v2] Calibrator = {calibrator}")
    print(f"[VAL] Threshold* = {t_star:.6f} | KS* = {ks_star:.4f}")

    for name, m in [("TRAIN", m_train), ("VAL", m_val), ("TEST", m_test)]:
        print(f"\n=== {name} ===")
        print(f"KS={m['KS']:.4f} | AUROC={m['AUROC']:.4f}")
        print(f"CE={m['CrossEntropy']:.6f} | MSE={m['MSE']:.6f}")
        print(f"Precision={m['precision']:.4f} | Recall={m['recall']:.4f} | F1={m['f1']:.4f}")
        print("Confusion matrix:\n", m["confusion_matrix"])

    return {
        "threshold_star": t_star,
        "ks_star": ks_star,
        "train": m_train,
        "val": m_val,
        "test": m_test,
        "proba": {"train": p_tr, "val": p_va, "test": p_te},
        "y": {"train": ytr, "val": yva, "test": yte}
    }






In [ ]:
# 1) Baseline
res_none = run_tabpfn2(
    X_train, y_train,
    X_val, y_val,
    X_test, y_test,
    calibrator="none"
)

# 2) Calibração por Cross-Entropy
res_ce = run_tabpfn2(
    X_train, y_train,
    X_val, y_val,
    X_test, y_test,
    calibrator="platt"
)

# 3) Calibração por MSE
res_mse = run_tabpfn2(
    X_train, y_train,
    X_val, y_val,
    X_test, y_test,
    calibrator="mse"
)



[TabPFN v2] Calibrator = none
[VAL] Threshold* = 0.573730 | KS* = 0.3431

=== TRAIN ===
KS=0.3200 | AUROC=0.7066
CE=0.698863 | MSE=0.252855
Precision=0.6159 | Recall=0.8040 | F1=0.6975
Confusion matrix:
 [[1287 1294]
 [ 506 2075]]

=== VAL ===
KS=0.3431 | AUROC=0.7048
CE=0.699043 | MSE=0.252943
Precision=0.6252 | Recall=0.8490 | F1=0.7201
Confusion matrix:
 [[ 634  657]
 [ 195 1096]]

=== TEST ===
KS=0.3090 | AUROC=0.6916
CE=0.771916 | MSE=0.289076
Precision=0.3622 | Recall=0.8141 | F1=0.5013
Confusion matrix:
 [[620 671]
 [ 87 381]]

[TabPFN v2] Calibrator = platt
[VAL] Threshold* = 0.455121 | KS* = 0.3431

=== TRAIN ===
KS=0.3200 | AUROC=0.7066
CE=0.636519 | MSE=0.222511
Precision=0.6159 | Recall=0.8040 | F1=0.6975
Confusion matrix:
 [[1287 1294]
 [ 506 2075]]

=== VAL ===
KS=0.3431 | AUROC=0.7048
CE=0.637498 | MSE=0.222912
Precision=0.6252 | Recall=0.8490 | F1=0.7201
Confusion matrix:
 [[ 634  657]
 [ 195 1096]]

=== TEST ===
KS=0.3090 | AUROC=0.6916
CE=0.646437 | MSE=0.227263
Prec

# Conclusões:

Foram conduzidos três experimentos com o modelo TabPFN v2, variando exclusivamente a estratégia de calibração probabilística: ausência de calibração, calibração por Platt Scaling (minimização da cross-entropy) e calibração por regressão linear orientada à minimização do erro quadrático médio (MSE). Em todos os casos, o modelo base, os dados e o critério de decisão foram mantidos constantes, sendo o threshold definido no conjunto de validação por meio da maximização da estatística KS, o que permitiu isolar o impacto da métrica de calibração na qualidade das probabilidades estimadas.

No cenário sem calibração, o modelo apresentou desempenho discriminativo consistente entre treino, validação e teste, com valores de AUROC próximos de 0,70 e KS em torno de 0,30. O threshold ótimo obtido na validação foi 0,5737, associado a um KS de 0,3431. Apesar da boa capacidade de ordenação e do alto recall da classe positiva, observou-se degradação significativa da calibração no conjunto de teste, evidenciada pelo aumento da cross-entropy e do MSE, indicando que as probabilidades previstas tornam-se menos confiáveis fora da amostra.

A aplicação do Platt Scaling resultou em melhoria expressiva da calibração probabilística, sem alterar as métricas de discriminação. Os valores de KS e AUROC permaneceram inalterados, enquanto a cross-entropy e o MSE apresentaram reduções substanciais, especialmente no conjunto de teste. Esse resultado indica que a calibração baseada em cross-entropy é eficaz para ajustar as probabilidades sem comprometer a capacidade discriminativa do modelo.

De forma semelhante, a calibração orientada à minimização do MSE manteve inalteradas as métricas discriminativas e produziu o menor erro quadrático médio no conjunto de teste, ainda que com valores de cross-entropy levemente superiores aos obtidos com Platt Scaling. As matrizes de confusão e as métricas de precisão, recall e F1-score permaneceram idênticas entre os três experimentos, uma vez que o threshold de decisão, definido via KS, é invariável a transformações monotônicas das probabilidades.

Em síntese, os resultados demonstram que o TabPFN v2 possui capacidade discriminativa estável, porém requer calibração explícita para fornecer probabilidades confiáveis. A escolha entre Platt Scaling e calibração por MSE deve ser guiada pela métrica probabilística de interesse da aplicação final, sem impacto sobre o desempenho classificatório no ponto de decisão adotado.